# Linear Model Selection & Regularization

### Loading Libraries

In [2]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# Data Visualization
from matplotlib.pyplot import subplots

# StatsModel
from statsmodels .api import OLS

# Scikit-Learn
import sklearn. linear_model as skl
from sklearn. pipeline import Pipeline
from sklearn. decomposition import PCA
import sklearn. model_selection as skm
from sklearn. preprocessing import StandardScaler
from sklearn. cross_decomposition import PLSRegression

In [3]:
# ISLP - Custom Libraries
from ISLP import load_data
from functools import partial
from ISLP.models import ModelSpec as MS
from ISLP.models import \
(Stepwise ,
sklearn_selected ,
sklearn_selection_path )

In [3]:
!pip install l0bnb

# !pip install git+https://github.com/joshmaxey/l0bnb
from l0bnb import fit_path

ModuleNotFoundError: No module named 'l0bnb'

### Subset Selection Methods

#### Forward Selection

In [4]:
Hitters = load_data('Hitters')

np.isnan(Hitters['Salary']).sum()

np.int64(59)

In [5]:
Hitters = Hitters.dropna ();
Hitters.shape

(263, 20)

In [6]:
def nCp(sigma2, estimator, X, Y):
    "Negative Cp statistic"
    n, p = X.shape
    Yhat = estimator .predict(X)
    RSS = np.sum ((Y - Yhat)**2)
    return -(RSS + 2 * p * sigma2) / n

In [7]:
design = MS(Hitters.columns.drop('Salary')).fit(Hitters)

Y = np.array(Hitters['Salary'])
X = design. transform (Hitters)

sigma2 = OLS(Y, X).fit().scale

In [8]:
neg_Cp = partial(nCp , sigma2)

In [9]:
strategy = Stepwise . first_peak (design ,
direction ='forward ',
max_terms =len(design.terms))

In [10]:
hitters_MSE = sklearn_selected(OLS, strategy )
hitters_MSE.fit(Hitters, Y)

hitters_MSE.selected_state_

('Assists',
 'AtBat',
 'CAtBat',
 'CHits',
 'CHmRun',
 'CRBI',
 'CRuns',
 'CWalks',
 'Division',
 'Errors',
 'Hits',
 'HmRun',
 'League',
 'NewLeague',
 'PutOuts',
 'RBI',
 'Runs',
 'Walks',
 'Years')

In [12]:
hitters_Cp = sklearn_selected (OLS, strategy, scoring=neg_Cp)

hitters_Cp.fit(Hitters, Y)
hitters_Cp.selected_state_

('Assists',
 'AtBat',
 'CAtBat',
 'CRBI',
 'CRuns',
 'CWalks',
 'Division',
 'Hits',
 'PutOuts',
 'Walks')

In [ ]:
#### Choosing Among Models Using the Validation Set Approach & Cross-Validation